In [1]:
import datawig
import pandas as pd
from google.colab import files
import io
import statistics
import numpy as np

In [2]:
uploaded = files.upload()

Saving trainSet.txt to trainSet (2).txt


In [3]:
data = pd.read_csv(io.StringIO(uploaded["trainSet.txt"].decode("utf-8")), sep=",")

In [4]:
data = data.set_index("PatientID")
#data["Sex"] = data["Sex"].astype(str)
#data["Cough"] = data["Cough"].astype(str)
#data["DifficultyInBreathing"] = data["DifficultyInBreathing"].astype(str)
#data["RespiratoryFailure"] = data["RespiratoryFailure"].astype(str)
#data["CardiovascularDisease"] = data["CardiovascularDisease"].astype(str)

In [5]:
data

,ImageFile,Hospital,Age,Sex,Temp_C,Cough,DifficultyInBreathing,WBC,CRP,Fibrinogen,LDH,Ddimer,Ox_percentage,PaO2,SaO2,pH,CardiovascularDisease,RespiratoryFailure,Prognosis
PatientID,,,,,,,,,,,,,,,,,,,
P_131,P_131.png,D,35.913889,0,39.3,1.0,0.0,5.76,43.40,651.0,387.0,157.0,94.0,NaN,NaN,NaN,0.0,NaN,MILD
P_132,P_132.png,D,57.266667,0,37.0,0.0,0.0,11.48,64.00,700.0,338.0,601.0,94.0,75.0,96.9,7.42,0.0,NaN,MILD
P_195,P_195.png,D,79.263889,0,37.8,1.0,0.0,6.21,115.30,698.0,356.0,448.0,94.0,63.0,94.6,7.39,1.0,NaN,SEVERE
P_193,P_193.png,D,82.000000,0,38.0,1.0,0.0,7.28,149.30,513.0,482.0,NaN,97.0,68.0,96.3,7.46,0.0,NaN,SEVERE
P_140,P_140.png,D,60.791667,1,37.0,1.0,0.0,6.37,20.70,NaN,NaN,210.0,93.0,NaN,97.3,NaN,0.0,NaN,MILD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P_1_12,P_1_12.png,F,51.000000,0,NaN,0.0,1.0,14.30,22.79,550.0,368.0,5027.0,NaN,NaN,NaN,NaN,0.0,0.0,SEVERE
P_1_8,P_1_8.png,F,57.000000,0,NaN,1.0,0.0,5.10,8.93,757.0,451.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,SEVERE
P_1_10,P_1_10.png,F,38.000000,0,NaN,0.0,1.0,7.30,0.23,NaN,NaN,1073.0,NaN,NaN,NaN,NaN,0.0,0.0,MILD


In [6]:
data.pop("ImageFile")
data.pop("Hospital")
data.pop("Prognosis")

PatientID
P_131        MILD
P_132        MILD
P_195      SEVERE
P_193      SEVERE
P_140        MILD
            ...  
P_1_12     SEVERE
P_1_8      SEVERE
P_1_10       MILD
P_1_26     SEVERE
P_1_146      MILD
Name: Prognosis, Length: 863, dtype: object

In [7]:
data_stdiv = data.std()

In [8]:
data_stdiv

Age                        15.052354
Sex                         0.473027
Temp_C                      0.970634
Cough                       0.500225
DifficultyInBreathing       0.500283
WBC                         3.532203
CRP                        66.927036
Fibrinogen                158.364728
LDH                       235.221008
Ddimer                   6743.128329
Ox_percentage               7.000610
PaO2                       26.114734
SaO2                        8.238277
pH                          0.056283
CardiovascularDisease       0.447907
RespiratoryFailure          0.124108
dtype: float64

In [9]:
df_train, df_test = datawig.utils.random_split(data)

In [10]:
def score(data_in):
  names = data_in.columns
  imputed_data = data_in[names[-1]]
  gt_name = names[-1][0:-8]
  ground_truth = data_in[gt_name]
  score = (((imputed_data-ground_truth)/ data_stdiv[gt_name])**2)
  score_1 = []
  for i, value in enumerate(list(score)):
    if not np.isnan(value):
      score_1.append(min([value, 1]))

  score = statistics.mean(score_1)

  return score

In [11]:
score_all = []
imputed_data = pd.DataFrame()
categoricals = ["Sex", "Cough", "DifficultyInBreathing", "RespiratoryFailure", "CardiovascularDisease"]

for i in range(len(data.columns)):
  print(i)
  input_columns = list(data.columns)
  input_columns.pop(i)
  output_columns = data.columns[i]
  
  imputer = datawig.SimpleImputer(
      input_columns=input_columns, # column(s) containing information about the column we want to impute
      output_column= output_columns, # the column we'd like to impute values for
      )
  imputer.fit(train_df=df_train, num_epochs=50)
  imputed = imputer.predict(df_test)
  imputed_train = imputer.predict(data)
  if output_columns in categoricals:
    imputed[imputed.columns[-1]] = imputed[imputed.columns[-1]].astype(int)
    imputed_train[imputed_train.columns[-1]] = imputed_train[imputed_train.columns[-1]].astype(int)
  imputed_data[output_columns] = imputed_train[imputed_train.columns[-1]]
  score_all.append(score(imputed))

0


2021-04-28 16:08:36,026 [INFO]  
========== start: fit model
2021-04-28 16:08:36,029 [WARNING]  Already bound, ignoring bind()
2021-04-28 16:08:36,066 [INFO]  Epoch[0] Batch [0-20]	Speed: 11314.36 samples/sec	cross-entropy=14.350975	Age-accuracy=0.000000
2021-04-28 16:08:36,099 [INFO]  Epoch[0] Train-cross-entropy=13.515664
2021-04-28 16:08:36,101 [INFO]  Epoch[0] Train-Age-accuracy=0.000000
2021-04-28 16:08:36,103 [INFO]  Epoch[0] Time cost=0.069
2021-04-28 16:08:36,112 [INFO]  Saved checkpoint to "age/model-0000.params"
2021-04-28 16:08:36,121 [INFO]  Epoch[0] Validation-cross-entropy=13.678425
2021-04-28 16:08:36,125 [INFO]  Epoch[0] Validation-Age-accuracy=0.000000
2021-04-28 16:08:36,161 [INFO]  Epoch[1] Batch [0-20]	Speed: 11221.84 samples/sec	cross-entropy=11.498455	Age-accuracy=0.000000
2021-04-28 16:08:36,189 [INFO]  Epoch[1] Train-cross-entropy=11.378387
2021-04-28 16:08:36,190 [INFO]  Epoch[1] Train-Age-accuracy=0.000000
2021-04-28 16:08:36,197 [INFO]  Epoch[1] Time cost=0.0

1
2


/usr/local/lib/python3.7/dist-packages/datawig/calibration.py:92: RuntimeWarning: invalid value encountered in log
  return np.log(probas)


3
4
5
6
7


/usr/local/lib/python3.7/dist-packages/datawig/calibration.py:92: RuntimeWarning: invalid value encountered in log
  return np.log(probas)


8
9
10
11
12
13
14
15


/usr/local/lib/python3.7/dist-packages/datawig/calibration.py:92: RuntimeWarning: invalid value encountered in log
  return np.log(probas)
/usr/local/lib/python3.7/dist-packages/datawig/calibration.py:92: RuntimeWarning: invalid value encountered in log
  return np.log(probas)


In [12]:
score_all

[0.4799175278977462,
 0.36627906976744184,
 0.5156706367446574,
 0.5116279069767442,
 0.47093023255813954,
 0.46316853237788536,
 0.34166156779847656,
 0.4150217872311786,
 0.30055195859163264,
 0.23045086766351514,
 0.34948065826608576,
 0.31126989219888795,
 0.30828362367551787,
 0.40003566667709756,
 0.23809523809523808,
 0.013888888888888888]

In [13]:
imputed_data

,Age,Sex,Temp_C,Cough,DifficultyInBreathing,WBC,CRP,Fibrinogen,LDH,Ddimer,Ox_percentage,PaO2,SaO2,pH,CardiovascularDisease,RespiratoryFailure
PatientID,,,,,,,,,,,,,,,,
P_131,51.692888,0,38.132470,1,0,7.383656,53.635326,597.832917,302.067710,1734.230170,93.769091,60.826678,90.532035,7.425381,0,0
P_132,63.693653,0,37.687642,0,0,8.784117,45.883545,616.881492,309.250699,834.942993,93.920856,66.792136,94.154430,7.453023,0,0
P_195,73.742816,0,37.958825,0,0,8.245370,90.009535,606.624214,315.412725,-46.030357,91.496674,69.114238,92.794176,7.451292,0,0
P_193,60.016046,0,37.811389,0,0,7.465410,109.835247,622.394668,293.676340,2961.654831,92.870065,84.069843,96.039631,7.446876,0,0
P_140,59.632027,0,37.811896,0,0,6.850413,53.000610,600.624542,294.877142,3144.166879,95.176803,77.924332,93.017696,7.464693,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P_1_12,64.254067,0,37.526832,0,0,8.293942,25.416685,578.384507,579.990908,3849.644928,91.441921,73.433792,90.964558,7.457718,0,0
P_1_8,56.996852,0,37.872865,0,0,9.254016,27.076496,603.069686,351.868413,2551.947575,91.528817,64.668325,91.228967,7.426213,0,0
P_1_10,59.234502,0,37.720409,0,0,6.814539,5.998739,552.784460,365.103507,1750.220776,92.507108,70.581495,91.937524,7.445044,0,0


In [14]:
data

,Age,Sex,Temp_C,Cough,DifficultyInBreathing,WBC,CRP,Fibrinogen,LDH,Ddimer,Ox_percentage,PaO2,SaO2,pH,CardiovascularDisease,RespiratoryFailure
PatientID,,,,,,,,,,,,,,,,
P_131,35.913889,0,39.3,1.0,0.0,5.76,43.40,651.0,387.0,157.0,94.0,NaN,NaN,NaN,0.0,NaN
P_132,57.266667,0,37.0,0.0,0.0,11.48,64.00,700.0,338.0,601.0,94.0,75.0,96.9,7.42,0.0,NaN
P_195,79.263889,0,37.8,1.0,0.0,6.21,115.30,698.0,356.0,448.0,94.0,63.0,94.6,7.39,1.0,NaN
P_193,82.000000,0,38.0,1.0,0.0,7.28,149.30,513.0,482.0,NaN,97.0,68.0,96.3,7.46,0.0,NaN
P_140,60.791667,1,37.0,1.0,0.0,6.37,20.70,NaN,NaN,210.0,93.0,NaN,97.3,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P_1_12,51.000000,0,NaN,0.0,1.0,14.30,22.79,550.0,368.0,5027.0,NaN,NaN,NaN,NaN,0.0,0.0
P_1_8,57.000000,0,NaN,1.0,0.0,5.10,8.93,757.0,451.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0
P_1_10,38.000000,0,NaN,0.0,1.0,7.30,0.23,NaN,NaN,1073.0,NaN,NaN,NaN,NaN,0.0,0.0


In [15]:
new_data = pd.DataFrame()
for i in range(data.shape[1]):
  column = data.columns[i]
  indexes_nan = np.argwhere(np.isnan(data[column]))
  new_data_column = data[column]
  imputed_data_column = imputed_data[column]
  new_data_column[np.isnan(new_data_column)] = imputed_data_column[np.isnan(new_data_column)]
  new_data[column] = new_data_column

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:58: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [16]:
new_data

,Age,Sex,Temp_C,Cough,DifficultyInBreathing,WBC,CRP,Fibrinogen,LDH,Ddimer,Ox_percentage,PaO2,SaO2,pH,CardiovascularDisease,RespiratoryFailure
PatientID,,,,,,,,,,,,,,,,
P_131,35.913889,0,39.300000,1.0,0.0,5.76,43.40,651.000000,387.000000,157.000000,94.000000,60.826678,90.532035,7.425381,0.0,0.0
P_132,57.266667,0,37.000000,0.0,0.0,11.48,64.00,700.000000,338.000000,601.000000,94.000000,75.000000,96.900000,7.420000,0.0,0.0
P_195,79.263889,0,37.800000,1.0,0.0,6.21,115.30,698.000000,356.000000,448.000000,94.000000,63.000000,94.600000,7.390000,1.0,0.0
P_193,82.000000,0,38.000000,1.0,0.0,7.28,149.30,513.000000,482.000000,2961.654831,97.000000,68.000000,96.300000,7.460000,0.0,0.0
P_140,60.791667,1,37.000000,1.0,0.0,6.37,20.70,600.624542,294.877142,210.000000,93.000000,77.924332,97.300000,7.464693,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P_1_12,51.000000,0,37.526832,0.0,1.0,14.30,22.79,550.000000,368.000000,5027.000000,91.441921,73.433792,90.964558,7.457718,0.0,0.0
P_1_8,57.000000,0,37.872865,1.0,0.0,5.10,8.93,757.000000,451.000000,2551.947575,91.528817,64.668325,91.228967,7.426213,0.0,0.0
P_1_10,38.000000,0,37.720409,0.0,1.0,7.30,0.23,552.784460,365.103507,1073.000000,92.507108,70.581495,91.937524,7.445044,0.0,0.0
